In [1]:
import pandas as pd
import numpy as np

from hyperopt.pyll import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

import lightgbm as lgb
from sklearn.metrics import (accuracy_score, f1_score, 
                             precision_score, recall_score, roc_auc_score)


import warnings
warnings.filterwarnings('ignore')

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Attrition")

<Experiment: artifact_location='/home/godwin/Documents/Workflow/Attriton/notebooks/mlruns/1', creation_time=1701070848011, experiment_id='1', last_update_time=1701070848011, lifecycle_stage='active', name='Attrition', tags={}>

In [24]:
train_data = pd.read_csv('../data/bct-data-summit/train.csv')
test_data = pd.read_csv('../data/bct-data-summit/test.csv')

numerical_col = train_data.select_dtypes(exclude=['object']).columns.tolist()
numerical_col.remove('id')
numerical_col.remove( 'Attrition')
numerical_col.remove('EmployeeCount')
numerical_col.remove('StandardHours')

categorical_col = train_data.select_dtypes(include=['object']).columns.tolist()
categorical_col.remove('Over18')

In [5]:
# categorical_col = ['BusinessTravel', 'Department', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime','newage', 'masterylevel', 'loyaltylevel', 'oldyoung', 'loyal']
# numerical_col = ['DailyRate', 'DistanceFromHome',  'Education',  'EnvironmentSatisfaction',
#             'HourlyRate', 'JobInvolvement', 'JobSatisfaction',  'MonthlyIncome',  'NumCompaniesWorked', 'PerformanceRating',
#             'RelationshipSatisfaction',  'StockOptionLevel',  'TrainingTimesLastYear',  'WorkLifeBalance',]

In [28]:
train_df, test_df = train_test_split(train_data, test_size = 0.25, random_state=0)
train_y, test_y = train_df.pop('Attrition'), test_df.pop("Attrition")

In [29]:
vectorizer = DictVectorizer()

train_dicts = train_df[categorical_col + numerical_col].to_dict(orient='records')
val_dicts = test_df[categorical_col + numerical_col].to_dict(orient='records')

vectorizer.fit(train_dicts)
feature_names = vectorizer.get_feature_names_out().tolist()

X_train = vectorizer.transform(train_dicts)
X_val = vectorizer.transform(val_dicts)

In [30]:
dtrain = lgb.Dataset(X_train, label = train_y, free_raw_data=False)
dtest = lgb.Dataset(X_val, label = test_y, reference=dtrain,
                    free_raw_data=False)

In [31]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.set_tag("loss", "aroc")
        mlflow.log_params(params)
        booster = lgb.train(
            params=params,
            train_set=dtrain, 
            num_boost_round=1000,
            valid_sets=[dtest],
            )
        prediction0 = booster.predict(X_val)
        prediction = (prediction0 >= 0.5).astype('int')
        aroc = roc_auc_score(test_y, prediction0)
        output = {"acc": accuracy_score(test_y, prediction), "f1_score": f1_score(test_y, prediction), 
                    "precision": precision_score(test_y, prediction), "recall": recall_score(test_y, prediction),
                    "area_roc":aroc}
        mlflow.log_metrics(output)

    return {'loss': -aroc, 'status': STATUS_OK}

In [32]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'boosting': 'gbdt',
    'num_iterations': 120,
    'num_leaves': scope.int(hp.quniform('num_leaves', 4, 100, 1)),
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000179 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1156                     
[LightGBM] [Info] Number of data points in the train set: 1257, number of used features: 50
[LightGBM] [Info] Start training from score 0.119332  
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

In [44]:
from imblearn.under_sampling import RandomUnderSampler

In [45]:
new_train_data = train_data.copy()

undersample = RandomUnderSampler(sampling_strategy=0.4, random_state = 0)
y = new_train_data.pop('Attrition')
X_train_new, y = undersample.fit_resample(new_train_data, y)
train_x, test_x,train_y, test_y = train_test_split(X_train_new, y, test_size = 0.25, random_state=0)

In [46]:
vectorizer = DictVectorizer()

train_dicts = train_x[categorical_col + numerical_col].to_dict(orient='records')
val_dicts = test_x[categorical_col + numerical_col].to_dict(orient='records')

vectorizer.fit(train_dicts)
feature_names = vectorizer.get_feature_names_out().tolist()

X_train = vectorizer.transform(train_dicts)
X_val = vectorizer.transform(val_dicts)

dtrain = lgb.Dataset(X_train, label = train_y)
dtest = lgb.Dataset(X_val, label = test_y)

In [47]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.set_tag("data", "full")
        mlflow.set_tag("scaler", "minmax")
        mlflow.set_tag("sampling", 'undersampling')
        mlflow.log_params(params)
        booster = lgb.train(
                params=params,
                train_set=dtrain, 
                num_boost_round=1000,
                valid_sets=[dtest],
                )
        prediction = booster.predict(X_val)
        prediction_label = (prediction >= 0.5).astype('int')
        aroc = roc_auc_score(test_y, prediction)
        output = {"acc": accuracy_score(test_y, prediction_label), "f1_score": f1_score(test_y, prediction_label), 
                        "precision": precision_score(test_y, prediction_label), "recall": recall_score(test_y, prediction_label), "area_roc":aroc}
        mlflow.log_metrics(output)

    return {'loss': -aroc, 'status': STATUS_OK}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007886 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 884                      
[LightGBM] [Info] Number of data points in the train set: 525, number of used features: 47
[LightGBM] [Info] Start training from score 0.289524  
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-t

In [49]:
with mlflow.start_run():
    
    train = lgb.Dataset(X_train, label=train_y)
    valid = lgb.Dataset(X_val, label=test_y)

    best_params = {"boosting":"gbdt",
                "learning_rate":best_result['learning_rate'],
                "max_depth":int(best_result['max_depth']),
                "num_iterations":120,
                "num_leaves":int(best_result['num_leaves']),
                "seed":42}

    booster = lgb.train(
        params=best_params,
        train_set=train,
        num_boost_round=1000,
        valid_sets=[valid],
    )

    prediction0 = booster.predict(X_val)
    prediction = (prediction0 >=0.5).astype('int')
    f1 = f1_score(test_y, prediction)
    output = {"acc": accuracy_score(test_y, prediction), "f1_score": f1, 
                "precision": precision_score(test_y, prediction), "recall": recall_score(test_y, prediction), 'area_roc':roc_auc_score(test_y, prediction0)}

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 884
[LightGBM] [Info] Number of data points in the train set: 525, number of used features: 47
[LightGBM] [Info] Start training from score 0.289524


In [50]:
output

{'acc': 0.8342857142857143,
 'f1_score': 0.6741573033707865,
 'precision': 0.7317073170731707,
 'recall': 0.625,
 'area_roc': 0.8841863517060367}

In [51]:
test_data = pd.read_csv('../data/bct-data-summit/test.csv')

test_data['newage'] = pd.cut(x = test_data['Age'], bins = [17, 30, 42, 61 ], labels = ['18 - 30', '31 - 42', '43 - 60'])
test_data['oldyoung'] = pd.cut(x = test_data['Age'], bins = [17, 30, 61], labels = ['young', 'old'])
test_data['loyal'] = pd.cut(x = test_data['YearsAtCompany'], bins = [-1, 3, 42], labels = ['fairly', 'loyal'])
test_data['masterylevel'] = pd.cut(x = test_data['TotalWorkingYears'], bins = [-1, 3, 10, 421], labels = ['entry', 'intermediate', 'master'])
test_data['loyaltylevel'] = pd.cut(x = test_data['YearsAtCompany'], bins = [-1, 3, 10, 42], labels = ['fairly', 'loyal', 'very-loyal'])
test_data['dueforprom'] = pd.cut(x = test_data['YearsSinceLastPromotion'], bins = [-1, 5,  16], labels = ['due', 'overdue'])


In [52]:
test_dat  = test_data[categorical_col + numerical_col].to_dict(orient='record')
X_test = vectorizer.transform(test_dat)

prediction = booster.predict(X_test).round(2)
dicts = {'id': test_data['id'], 'Attrition': prediction}
output_frame = pd.DataFrame(dicts)

(output_frame['Attrition'] >=0.5).astype('int').sum()

186

In [53]:
output_frame.to_csv('../submissions/lgb002.csv', index = False)